<a href="https://colab.research.google.com/github/ValentinKahn/KaggleSalesPrediction/blob/master/Kaggle_Sales_Prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [92]:
import io
import pandas as pd

df_itemcat = pd.read_csv('https://raw.githubusercontent.com/ValentinKahn/KaggleSalesPrediction/main/item_categories.csv')
df_item = pd.read_csv('https://raw.githubusercontent.com/ValentinKahn/KaggleSalesPrediction/main/items.csv')
df_train = pd.read_csv('https://raw.githubusercontent.com/ValentinKahn/KaggleSalesPrediction/main/sales_train.csv')
df_sample = pd.read_csv('https://raw.githubusercontent.com/ValentinKahn/KaggleSalesPrediction/main/sample_submission.csv')
df_shops = pd.read_csv('https://raw.githubusercontent.com/ValentinKahn/KaggleSalesPrediction/main/shops.csv')
df_test = pd.read_csv('https://raw.githubusercontent.com/ValentinKahn/KaggleSalesPrediction/main/test.csv')

In [93]:
df_train.insert(0,'ID',df_train.index)
df_item = df_item.join(df_itemcat.set_index('item_category_id'), on='item_category_id')
df_train = df_train.join(df_shops.set_index('shop_id'), on='shop_id')
df_train = df_train.join(df_item.set_index('item_id'), on='item_id')

In [94]:
import numpy as np
median = df_train.loc[df_train['item_price']<5000, 'item_price'].median()
df_train['item_price'] = np.where(df_train['item_price'] > 5000, median,df_train['item_price'])

In [95]:
df_train['month'] = pd.DatetimeIndex(df_train['date']).month

In [96]:
df_train = df_train.merge(df_train.groupby(['month','shop_id', 'item_id','item_price'])['item_cnt_day'].agg(['sum']), 
         left_on=['month','shop_id', 'item_id','item_price'], 
         right_index=True)

In [97]:
df_train = df_train.drop_duplicates(subset=['month','shop_id','item_id','item_price'], keep='first')

In [98]:
df_train = df_train.drop(['date','date_block_num','shop_name','item_name','item_category_name','item_cnt_day','month'], axis = 1)

In [99]:
df_train['shop_id'] = df_train.shop_id.astype(float)
df_train['item_category_id'] = df_train.item_category_id.astype(float)
df_train['item_id'] = df_train.item_id.astype(float)

In [100]:
from sklearn import preprocessing
from numpy import asarray
df_train['item_price']= preprocessing.normalize(df_train[['item_price']])
df_train.head()

,ID,shop_id,item_id,item_price,item_category_id,sum
0,0,59.0,22154.0,1.0,37.0,1.0
1,1,25.0,2552.0,1.0,58.0,1.0
2,2,25.0,2552.0,1.0,58.0,-1.0
3,3,25.0,2554.0,1.0,58.0,1.0
4,4,25.0,2555.0,1.0,56.0,1.0


In [101]:
from sklearn import datasets, ensemble
from sklearn.inspection import permutation_importance
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [106]:
X = df_train.iloc[:,:5]
y = df_train['sum']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=13)

params = {'n_estimators': 50,
          'max_depth': 4,
          'min_samples_split': 5,
          'learning_rate': 0.01,
          'loss': 'ls'}

In [107]:
reg = ensemble.GradientBoostingRegressor(**params)
reg.fit(X_train, y_train)

mse = mean_squared_error(y_test, reg.predict(X_test))
print("The mean squared error (MSE) on test set: {:.4f}".format(mse))

The mean squared error (MSE) on test set: 79.6749


In [104]:
#df_test = df_test.fillna(method='bfill')
df_test.isna().sum()

ID         0
shop_id    0
item_id    0
dtype: int64

In [109]:
#uploading to Google Drive for further use
!pip install -U -q PyDrive
import scipy.sparse as sparse
from pydrive.drive import GoogleDrive
from pydrive.auth import GoogleAuth
from google.colab import auth, files
from oauth2client.client import GoogleCredentials
model = reg
model.fit(X_train.iloc[:,:3],y_train)
output = model.predict(df_test).astype(float)
df_output = pd.DataFrame()
aux = df_test
df_output['ID'] = aux['ID']
df_output['item_cnt_month'] = output
df_output[['ID', 'item_cnt_month']].to_csv('./valentin_submission.csv', index=False)
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
uploaded = drive.CreateFile({'title': 'valentin_submission.csv'})
uploaded.SetContentFile('valentin_submission.csv')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1h3e6pRqqt8BB5itRRnpoq9PToAE2DQpx
